In [2]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [3]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [4]:
%matplotlib notebook

In [5]:
def to_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'\d+', cell)
    if _:
        return int(_[0])


df = pd.read_excel("赛题1数据材料/赛题1数据集/购地-地块公示.xlsx", index_col="企业编号")
df['出让年限'] = df['出让年限'].map(to_number)
df

,行政区,时间,公示日期,土地用途,土地面积（公顷）,出让年限,成交价（万元）,土地使用条件
企业编号,,,,,,,,
1001,内蒙古->呼伦贝尔市->海拉尔区,2010-07-14,2010年07月14日至2010年07月24日,商务金融用地,0.018960,40.0,56.0000,NaN
1001,内蒙古->呼伦贝尔市->海拉尔区,2010-07-14,2010年07月14日至2010年07月24日,商务金融用地,0.012060,40.0,40.0000,NaN
1001,内蒙古->呼伦贝尔市->海拉尔区,2010-07-14,2010年07月14日至2010年07月24日,商务金融用地,0.049570,40.0,136.0000,NaN
1001,湖南省 > 长沙市 > 长沙市本级,2012-11-20,2012年11月20日至2012年12月20日,商务金融用地,0.762853,40.0,10299.0000,"1、该宗地原于2009年1月划拨给长沙市滨江新城建设开发有限责任公司作为基础设施用地,其中"
1001,湖南省 > 长沙市 > 岳麓区,2012-11-26,2012年11月26日至2012年12月03日,商务金融用地,0.762853,40.0,10299.0000,"根据《长沙市城乡规划局规划设计条件书》(案卷编号20120172XDGH1),用途为商业金"
...,...,...,...,...,...,...,...,...
3995,广东省 > 深圳市 > 深圳市本级,2011-04-13,2011年04月13日至2011年04月15日,工业用地,3.895493,50.0,2842.3500,NaN
3998,鼓楼区,2015-10-08,2015年10月08日至2015年10月13日,商务金融用地,0.010798,40.0,156.2147,现状
3998,江苏省->南京市本级,2007-09-20,NaN,采矿用地,0.670000,NaN,NaN,NaN


In [6]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [7]:
time = df[['时间']].applymap(to_day).groupby('企业编号').mean().T
time = time.reindex(columns=list(range(1001, 4001)))
time

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
时间,-2704.971275,NaN,-2820.0,-1131.0,-501.0,NaN,-2238.75,-2137.555556,-2432.875,-2645.398448,...,-2399.0,NaN,-3258.666667,-2721.0,-2883.0,NaN,NaN,-2714.0,-2502.0,-1988.0


In [8]:
lst = df['土地用途'].value_counts()[df['土地用途'].value_counts() == 1].index


def to_land_use(cell):
    if not isinstance(cell, str):
        return cell
    if cell not in lst:
        return cell
    
    
land_use = pd.get_dummies(df['土地用途'].map(to_land_use), prefix='土地用途').groupby('企业编号').sum().T
land_use = land_use.reindex(columns=list(range(1001, 4001))).fillna(0)
land_use

企业编号,1001,1003,1004,1005,1007,1008,1009,1010,1011,1012,...,3987,3988,3989,3991,3993,3994,3995,3998,3999,4000
土地用途_05,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_05、07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
土地用途_街巷用地,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_采矿用地,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
土地用途_铁路用地,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_风景名胜设施用地,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
land_area_hectare = df[['土地面积（公顷）']].groupby('企业编号').sum().T
land_area_hectare = land_area_hectare.reindex(columns=list(range(1001, 4001)))
land_area_hectare

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
土地面积（公顷）,158135.998846,NaN,55.6681,4.0,2.93525,NaN,230.110791,232.8353,63.3853,66387.293,...,3.176779,NaN,7.3577,10.11,3.895493,NaN,NaN,0.680798,6.14196,1.0588


In [9]:
transfer_period = df[['出让年限']].groupby('企业编号').mean().T
transfer_period = transfer_period.reindex(columns=list(range(1001, 4001)))
transfer_period

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
出让年限,41.428571,NaN,50.0,NaN,50.0,NaN,50.0,50.0,40.0,NaN,...,50.0,NaN,NaN,50.0,50.0,NaN,NaN,40.0,50.0,50.0


In [10]:
transaction_price_ten_thousand_yuan = df[['成交价（万元）']].groupby('企业编号').sum().T
transaction_price_ten_thousand_yuan = transaction_price_ten_thousand_yuan.reindex(columns=list(range(1001, 4001)))
transaction_price_ten_thousand_yuan

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
成交价（万元）,738353.0,NaN,18741.7538,0.0,500.0,NaN,18721.83,21543.0652,1920.0,0.0,...,1068.0,NaN,0.0,3400.0,2842.35,NaN,NaN,156.2147,2064.0,397.05


In [11]:
pd.concat([time, land_use, land_area_hectare, transfer_period, transaction_price_ten_thousand_yuan]).to_pickle('购地-地块公示.pickle')